In [1]:
!nvidia-smi

Fri Jul 23 10:43:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install datasets transformers rouge-score nltk py7zr
! pip install wandb -qq

     |████████████████████████████████| 542 kB 8.5 MB/s 
     |████████████████████████████████| 2.6 MB 69.1 MB/s 
     |████████████████████████████████| 65 kB 2.3 MB/s 
     |████████████████████████████████| 76 kB 6.2 MB/s 
     |████████████████████████████████| 118 kB 75.7 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 243 kB 59.2 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.8 MB/s 
     |████████████████████████████████| 636 kB 49.2 MB/s 
     |████████████████████████████████| 121 kB 65.7 MB/s 
     |████████████████████████████████| 1.9 MB 55.5 MB/s 
     |████████████████████████████████| 2.2 MB 44.8 MB/s 
     |████████████████████████████████| 357 kB 56.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existin

In [4]:
import wandb
wandb.login()

wandb: Currently logged in as: nid989 (use `wandb login --relogin` to force relogin)


True

In [5]:
%env WANDB_PROJECT=bart_large_xsum_samsum

env: WANDB_PROJECT=bart_large_xsum_samsum


In [6]:
model_checkpoint = "facebook/bart-large-xsum"

## Loading the dataset

In [7]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("samsum")
metric = load_metric("rouge")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/770 [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

## Preprocessing the data

In [8]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

## Fine-tuning the model

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [12]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",  # enable logging to W&B
    run_name="bart-large-xsum-samsum"  # name of the W&B run (optional)
)

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [16]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, dialogue, id.
***** Running training *****
  Num examples = 14732
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 5523
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.384800,1.447523,53.373100,28.897700,44.133000,48.806400,25.792200
1,1.047500,1.478893,54.286800,29.193600,44.496500,49.723900,29.298300
2,0.856500,1.488593,54.532000,29.692300,45.149800,50.202500,30.198000


Saving model checkpoint to test-dialogue-summarization/checkpoint-500
Configuration saved in test-dialogue-summarization/checkpoint-500/config.json
Model weights saved in test-dialogue-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-dialogue-summarization/checkpoint-1000
Configuration saved in test-dialogue-summarization/checkpoint-1000/config.json
Model weights saved in test-dialogue-summarization/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-dialogue-summarization/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-dialogue-summarization/checkpoint-1000/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; 

TrainOutput(global_step=5523, training_loss=1.116441732900807, metrics={'train_runtime': 5549.3066, 'train_samples_per_second': 7.964, 'train_steps_per_second': 0.995, 'total_flos': 3.003043192720589e+16, 'train_loss': 1.116441732900807, 'epoch': 3.0})

In [18]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: summary, dialogue, id.
***** Running Evaluation *****
  Num examples = 818
  Batch size = 4


{'epoch': 3.0,
 'eval_gen_len': 30.198,
 'eval_loss': 1.4885928630828857,
 'eval_rouge1': 54.532,
 'eval_rouge2': 29.6923,
 'eval_rougeL': 45.1498,
 'eval_rougeLsum': 50.2025,
 'eval_runtime': 240.962,
 'eval_samples_per_second': 3.395,
 'eval_steps_per_second': 0.851}

In [19]:
wandb.finish()

train/loss,0.8565
train/learning_rate,0.0
train/epoch,3.0
train/global_step,5523
_runtime,5855
_timestamp,1627043108
_step,15
eval/loss,1.48859
eval/rouge1,54.532
eval/rouge2,29.6923
eval/rougeL,45.1498


train/loss,█▇▆▅▃▃▃▂▁▁▁
train/learning_rate,█▇▇▆▅▅▄▃▂▂▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▇▇████
train/global_step,▁▂▂▃▃▄▄▅▅▆▇▇████
_runtime,▁▂▂▃▃▄▄▅▅▅▆▇▇███
_timestamp,▁▂▂▃▃▄▄▅▅▅▆▇▇███
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval/loss,▁▆██
eval/rouge1,▁▇██
eval/rouge2,▁▄██
eval/rougeL,▁▄██


[Uploaded the model](https://huggingface.co/transformers/model_sharing.html) to the [🤗 Model Hub](https://huggingface.co/models). You can use it to generate results as shown below.

In [ ]:
import re

In [ ]:
x = """PERSON3: So like if you the keep like keeping the ajor focus on the suarization part.Like to to build suaries very close to how how the are.Should be the ain focus.And the in like u this (envisioned) to lose the evolution etrics that we are going to propose.Should coe in the second like u not in the fast shared task.Because that will be like then then we will ixing up too any things in one in one place.
PERSON5: ,well, I a not sure if understand all the details..
PERSON3): So I a saying like u little skip the fast shared task as siple as possibly.Like we have already soe evaluation etrics like and other evaluation etrics that we have.Okay and we also have a anualy evaluation think that is prepared by.And I think it is it is good that we can involve annotators to
PERSON5): But does has does have at least soe inial sense, doesn't give soe totaly unr unrealistic nubers that doesn't that don't say anything serious about the quality of the annotations..
PERSON6): says, but we are in a counity.
PERSON3): Yeah.It is it is for suarization the rules is that.You know like, yeahh.I know like we that that that.But it is the.
PERSON5): So actually, why we want to run this is tool is that has actually no real sense run estiating the suariz on the one hand,and on the other hand that anual etrix, anual evolutions that we have now are very very subjective and u kind of also very wake.And in these keys using the (alignent) could give us a little ore than when at the counity.
PERSON3): Yesss.Yes.
PERSON5): Probably if we don't put at in to the shared task, we kind of don't use
PERSON3): Yeah.
PERSON5): We could you.
PERSON3): It this investigation this desert should definitelly go in the background., for the alignent thing,because this this this could be on new contribution fro (all in).
PERSON5): Yeah."""
y = x.split('\n')
a = []
for Y in y:
  temp = re.sub('\(','',Y)
  temp = re.sub('\)','',Y)
  a.append(temp)

a

['PERSON3: So like if you the keep like keeping the ajor focus on the suarization part.Like to to build suaries very close to how how the are.Should be the ain focus.And the in like u this (envisioned to lose the evolution etrics that we are going to propose.Should coe in the second like u not in the fast shared task.Because that will be like then then we will ixing up too any things in one in one place.',
 'PERSON5: ,well, I a not sure if understand all the details..',
 'PERSON3: So I a saying like u little skip the fast shared task as siple as possibly.Like we have already soe evaluation etrics like and other evaluation etrics that we have.Okay and we also have a anualy evaluation think that is prepared by.And I think it is it is good that we can involve annotators to',
 "PERSON5: But does has does have at least soe inial sense, doesn't give soe totaly unr unrealistic nubers that doesn't that don't say anything serious about the quality of the annotations..",
 'PERSON6: says, but we 

In [ ]:
conversation_ = '\n'.join(a)
conversation_

"PERSON3: So like if you the keep like keeping the ajor focus on the suarization part.Like to to build suaries very close to how how the are.Should be the ain focus.And the in like u this (envisioned to lose the evolution etrics that we are going to propose.Should coe in the second like u not in the fast shared task.Because that will be like then then we will ixing up too any things in one in one place.\nPERSON5: ,well, I a not sure if understand all the details..\nPERSON3: So I a saying like u little skip the fast shared task as siple as possibly.Like we have already soe evaluation etrics like and other evaluation etrics that we have.Okay and we also have a anualy evaluation think that is prepared by.And I think it is it is good that we can involve annotators to\nPERSON5: But does has does have at least soe inial sense, doesn't give soe totaly unr unrealistic nubers that doesn't that don't say anything serious about the quality of the annotations..\nPERSON6: says, but we are in a coun

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")
conversation = '''Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye                                       
'''
summarizer(conversation_)

loading configuration file https://huggingface.co/lidiya/bart-large-xsum-samsum/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/74b1717d96d68d76efeb4180e6af68b4f41cff0eb22beafb3151a55fbe013515.812d5011531bf1e42ca2e012fc26a1eabb78ffe40a23629ab01328b0b488e639
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-xsum",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_i

[{'summary_text': "PERSON3 wants to keep the main focus on the suarization part. PERSON5 wants to skip the fast shared task. PERSON6 doesn't understand all the details. "}]